In [47]:
#Importar librerias
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer

In [365]:
# Importar csv que posee Consumo de energia y Temperatura de 23 provincias + CABA.
df = pd.read_csv("PBA.csv", header = 0, parse_dates = [0], index_col = 0, dayfirst=True)
df['Año'] = pd.DatetimeIndex(df.index).year
df['Mes'] = pd.DatetimeIndex(df.index).month
df['Trim'] = pd.DatetimeIndex(df.index).quarter
df.tail()

,Hora,Tipo Dia,Energia PBA,Clima PBA,Energia GBA,Año,Mes,Trim
Fecha,,,,,,,,
2021-07-31 19:00:00,19,7,NaN,9.566667,NaN,2021,7,3
2021-07-31 20:00:00,20,7,NaN,8.633333,NaN,2021,7,3
2021-07-31 21:00:00,21,7,NaN,7.700000,NaN,2021,7,3
2021-07-31 22:00:00,22,7,NaN,7.150000,NaN,2021,7,3
2021-07-31 23:00:00,23,7,NaN,6.600000,NaN,2021,7,3


In [366]:
#Calculo del rango intercuartil
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

Hora             11.50000
Tipo Dia          4.00000
Energia PBA     326.31875
Clima PBA         9.50000
Energia GBA    1706.54325
Año               2.00000
Mes               6.00000
Trim              2.00000
dtype: float64


In [367]:
#Reemplazar outliers con NaN utilizando el IQR

df = df.mask((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)), np.nan)
df.isna().sum()

Hora             0
Tipo Dia         0
Energia PBA    217
Clima PBA      103
Energia GBA    335
Año              0
Mes              0
Trim             0
dtype: int64

In [368]:
df1 = df

In [369]:
df1

,Hora,Tipo Dia,Energia PBA,Clima PBA,Energia GBA,Año,Mes,Trim
Fecha,,,,,,,,
2018-01-01 00:00:00,0,1,1544.873,19.200000,4593.287,2018,1,1
2018-01-01 01:00:00,1,1,1425.868,18.600000,4384.859,2018,1,1
2018-01-01 02:00:00,2,1,1387.791,18.500000,4266.546,2018,1,1
2018-01-01 03:00:00,3,1,1327.708,17.900000,4103.462,2018,1,1
2018-01-01 04:00:00,4,1,1268.554,17.500000,3912.347,2018,1,1
...,...,...,...,...,...,...,...,...
2021-07-31 19:00:00,19,7,NaN,9.566667,NaN,2021,7,3
2021-07-31 20:00:00,20,7,NaN,8.633333,NaN,2021,7,3
2021-07-31 21:00:00,21,7,NaN,7.700000,NaN,2021,7,3


In [370]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rmse = lambda y, yhat: np.sqrt(mean_squared_error(y, yhat))
def optimize_k(data, target):
    errors = []
    for k in range(1, 25, 2):
        imputer = KNNImputer(n_neighbors=k)
        imputed = imputer.fit_transform(data)
        df_imputed = pd.DataFrame(imputed, columns=df.columns)
        
        X = df_imputed.drop(target, axis=1)
        y = df_imputed[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        error = rmse(y_test, preds)
        errors.append({'K': k, 'RMSE': error})
        
    return errors

In [371]:
k_errors = optimize_k(data=df1, target='Energia PBA')

In [372]:
pd.DataFrame.from_dict(k_errors, orient='columns', dtype=None, columns=None)


,K,RMSE
0,1,59.756169
1,3,57.294862
2,5,59.323840
3,7,59.074579
4,9,57.092260
5,11,59.163016
6,13,58.118870
7,15,60.594999
8,17,58.551868
9,19,59.716713


In [348]:
# Completar los valores nulos utilizando KNN, con k=5

imputer = KNNImputer(n_neighbors=23)
df2 = pd.DataFrame(imputer.fit_transform(df1), columns = df.columns)

In [349]:
#Exportar csv sin outliers para el modelo
df2.to_csv("PBAfore.csv")